In [1]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import pandas as pd
import json
import numpy as np

from functions import (get_sector, changes_from_press, get_content_header,
                       get_df, normalise_to_index, save_data)


C:\Users\Aidan\miniconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = []


offset = 100

ticker = 'fslr'
sector = get_sector(ticker)

index_ticker = '^GSPC'

stock_hist = get_df(ticker)
index_hist = get_df(index_ticker)

year = datetime.today().year

years = np.arange(year, 2014, -1)

# create break
for year in years:
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"}
    
    s = requests.Session()
    html = requests.get(f'https://investor.firstsolar.com/news/{year}/default.aspx', headers=headers)
    

    soup = BeautifulSoup(html.content) 

    articles = soup.find_all('div',attrs={'class':'module_item'})
    
    if not articles:
        break
        
    for article in articles:
        date = article.find('span', attrs={'class':'module_date-text'}).text.lstrip()
        title = article.find('a', attrs={'class':'module_headline-link'}).text
        url = article.find('a', attrs={'class':'module_headline-link'}).get('href')
        
        date = date.lstrip().rstrip()    # remove starting and trailing whitespaces
        title = title.lstrip().rstrip() 
        
        date = date.replace(',', '')
        
        date = datetime.strptime(date, '%B %d %Y').date()
        
        pct_change = changes_from_press(stock_hist, date, 1)

        content = get_content_header(url, 'meta', {'name':'twitter:description'})
        
        if pct_change != None:
            index_price, norm_price = normalise_to_index(pct_change, date, index_hist)
        else:
            index_price, norm_price = (None, None)

        data.append([str(date), title, url, content, pct_change, index_price, norm_price])
    
    offset = offset + 100
    
save_data(data, ticker, sector)